In [2]:
"""
Purpose: To look at how the ratio of inhibiton/excitation
differ on different compartments

"""

'\nPurpose: To look at how the ratio of inhibiton/excitation\ndiffer on different compartments\n\n'

In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
from os import sys
sys.path.append("/datasci_tools/datasci_tools")
sys.path.append("/../graph_tools/")
sys.path.append("/meshAfterParty/meshAfterParty/")

In [15]:
import seaborn as sns
import matplotlib.pyplot as plt
import networkx_utils as xu
import system_utils as su
cmap = sns.cm.rocket_r

# Getting the Data

In [7]:
G = su.decompress_pickle("../../Data/G_query_v6_filtered.pbz2")

In [8]:
# generating the edge distribution sheet

xu.print_node_edges_counts(G)

import networkx_utils as xu
edge_df_ids=["presyn","postsyn"]
e_and_n_df,node_df = xu.edge_and_node_df(G,
                                edge_df_ids=edge_df_ids,
                                node_df_id="segment_split_id",
                                 verbose = False,
                                return_node_df = True)

 Graph: # of nodes = 57458, # of edges = 1904155


# Want to Analyze the distribution

In [10]:
edge_df= e_and_n_df

In [11]:
edge_df["postsyn_compartment"] = (
    edge_df["postsyn_compartment_coarse"].astype("str") + "_"
    + edge_df["postsyn_compartment_fine"].astype("str"))

edge_df["presyn_compartment"] = (
    edge_df["presyn_compartment_coarse"].astype("str")
)

In [12]:
edge_df

,presyn,postsyn,synapse_x,postsyn_spine_bouton,presyn_spine_bouton,presyn_soma_postsyn_soma_euclid_dist,proof_type,presyn_branch_idx,presyn_nucleus_id_x,postsyn_soma_euclid_dist,...,postsyn_dendrite_bbox_z_max,postsyn_dendrite_bbox_x_min_soma_relative,postsyn_dendrite_bbox_y_min_soma_relative,postsyn_dendrite_bbox_z_min_soma_relative,postsyn_dendrite_bbox_x_max_soma_relative,postsyn_dendrite_bbox_y_max_soma_relative,postsyn_dendrite_bbox_z_max_soma_relative,postsyn_u,postsyn_compartment,presyn_compartment
0,864691135571273253_0,864691135639004475_0,67464,head,nolabel,85773.086688,auto,NaN,26381,86203.648554,...,8.155345e+05,-27317.698690,-16406.142055,-17156.997441,38000.650958,88744.380757,7221.534723,864691135639004475_0,dendrite_basal,soma
1,864691136023767609_0,864691136965808334_0,70486,shaft,nolabel,73756.304137,auto,NaN,29852,76824.070186,...,8.252842e+05,-27839.612182,-193289.299807,-2403.913531,105702.668776,47346.829512,31080.323227,864691136965808334_0,dendrite_apicalshaft,soma
2,864691135366598130_0,864691135656048934_0,78739,nolabel,nonbouton,113237.958653,auto,4.0,126842,6686.863540,...,8.448430e+05,-9217.680923,-18111.218332,-3545.366347,-3488.680923,-6155.218332,3008.633653,864691135656048934_0,soma_nan,axon
3,864691135366598130_0,864691135656048934_0,79684,nolabel,nonbouton,113237.958653,auto,4.0,126842,4808.825220,...,8.448430e+05,-9217.680923,-18111.218332,-3545.366347,-3488.680923,-6155.218332,3008.633653,864691135656048934_0,soma_nan,axon
4,864691135366598130_0,864691135065014340_0,78928,shaft,nonbouton,110780.263730,auto,10.0,126842,9493.679582,...,8.402110e+05,-11214.111002,-47398.120823,-11449.179338,12803.888998,28009.879177,10563.820662,864691135065014340_0,dendrite_basal,axon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1904150,864691134949380354_0,864691135161106357_0,407256,shaft,nonbouton,58932.496842,auto,19.0,764975,7353.541732,...,1.086430e+06,-42432.286608,-51568.522214,-4041.694130,17367.713392,14442.477786,12508.305870,864691135161106357_0,dendrite_apicalshaft,axon
1904151,864691135212663680_0,864691136056344024_0,113396,shaft,nolabel,68368.097004,auto,NaN,132326,68221.935241,...,8.655082e+05,-89680.921994,-170945.239877,-141771.638569,172075.997586,121757.183484,129707.595081,864691136056344024_0,dendrite_basal,soma
1904152,864691136236746767_0,864691136050870771_0,236772,shaft,nolabel,31625.808448,auto,NaN,407443,25940.557280,...,7.807096e+05,-222787.888699,-152272.857644,-157765.025427,134159.410889,28147.937779,28381.278914,864691136050870771_0,dendrite_basal,soma
1904153,864691135809467852_0,864691136542471330_0,371868,shaft,nolabel,140100.014390,auto,NaN,678673,135984.737791,...,1.003646e+06,-170471.390784,-172479.075479,-121315.749207,190208.609216,6161.396107,94121.993531,864691136542471330_0,dendrite_basal,soma


In [31]:
edge_df_filt = edge_df.query("presyn_compartment_coarse != 'soma'")
len(edge_df_filt)

1891168

In [41]:
curr_pivot = edge_df_filt.pivot_table(
                    columns = ["presyn_cell_type",],
                    index = ["postsyn_compartment","postsyn_cell_type"],
                    values = "postsyn", #the value that will ultimately be concatenated
                   aggfunc = "count")
curr_pivot["percentage"] = curr_pivot["inhibitory"]/(curr_pivot["inhibitory"] + curr_pivot["excitatory"]) 
curr_pivot

# fig,ax = plt.subplots(1,figsize=(10,5))
# sns.heatmap(curr_pivot,cmap=cmap)

presyn_cell_type                        excitatory  inhibitory  percentage
postsyn_compartment  postsyn_cell_type                                    
axon_nan             excitatory                461       10088    0.956299
                     inhibitory                159         477    0.750000
dendrite_apical      excitatory              13770      117853    0.895383
dendrite_apicalshaft excitatory              11463      101309    0.898352
dendrite_apicaltuft  excitatory              10867       52460    0.828399
dendrite_basal       excitatory             132222      848406    0.865166
dendrite_nan         inhibitory             154689      109569    0.414629
dendrite_oblique     excitatory              22828      101517    0.816414
soma_nan             excitatory               1297      181044    0.992887
                     inhibitory               3370       17319    0.837112

# Checking Excitatory to Basal

In [79]:
import pandas_utils as pu
pu.set_pd_display_no_truncate_col()
edge_df_filt.query("(postsyn_compartment == 'dendrite_basal') and (presyn_cell_type == 'excitatory') " )[["presyn","postsyn","postsyn_cell_type"]]

edge_idx = 1
edge_df_filt.iloc[[edge_idx],:]

,presyn,postsyn,synapse_x,postsyn_spine_bouton,presyn_spine_bouton,presyn_soma_postsyn_soma_euclid_dist,proof_type,presyn_branch_idx,presyn_nucleus_id_x,postsyn_soma_euclid_dist,postsyn_limb_idx,synapse_y,presyn_skeletal_distance_to_soma,synapse_id,presyn_limb_idx,synapse_z,synapse_size,presyn_compartment_coarse,presyn_soma_postsyn_soma_skeletal_dist,postsyn_skeletal_distance_to_soma,presyn_soma_euclid_dist,postsyn_compartment_coarse,postsyn_nucleus_id_x,postsyn_branch_idx,presyn_compartment_fine,postsyn_compartment_fine,presyn_visual_area,presyn_layer,presyn_multiplicity,presyn_cell_type_used,presyn_cell_type,presyn_nucleus_id_y,presyn_nuclei_distance,presyn_n_nuclei_in_radius,presyn_n_nuclei_in_bbox,presyn_soma_x,presyn_soma_y,presyn_soma_z,presyn_soma_x_nm,presyn_soma_y_nm,presyn_soma_z_nm,presyn_max_soma_n_faces,presyn_max_soma_volume,presyn_max_soma_area,presyn_syn_density_post_after_proof,presyn_syn_density_head_after_proof,presyn_syn_density_neck_after_proof,presyn_syn_density_shaft_after_proof,presyn_skeletal_length_processed_syn_after_proof,presyn_spine_density_after_proof,presyn_skeletal_length_processed_spine_after_proof,presyn_baylor_e_i_after_proof,presyn_baylor_e_i,presyn_allen_e_i,presyn_cell_type_used_for_axon,presyn_cell_type_for_axon,presyn_allen_e_i_n_nuc,presyn_allen_cell_type,presyn_allen_cell_type_n_nuc,presyn_allen_cell_type_e_i,presyn_axon_angle_max,presyn_axon_angle_min,presyn_n_axon_angles,presyn_n_vertices,presyn_n_faces,presyn_n_not_processed_soma_containing_meshes,presyn_n_error_limbs,presyn_n_same_soma_multi_touching_limbs,presyn_n_multi_soma_touching_limbs,presyn_n_somas,presyn_n_limbs,presyn_n_branches,presyn_max_limb_n_branches,presyn_skeletal_length,presyn_max_limb_skeletal_length,presyn_median_branch_length,presyn_width_median,presyn_width_no_spine_median,presyn_width_90_perc,presyn_width_no_spine_90_perc,presyn_n_spines,presyn_n_boutons,presyn_spine_density,presyn_spines_per_branch,presyn_skeletal_length_eligible,presyn_n_spine_eligible_branches,presyn_spine_density_eligible,presyn_spines_per_branch_eligible,presyn_total_spine_volume,presyn_spine_volume_median,presyn_spine_volume_density,presyn_spine_volume_density_eligible,presyn_spine_volume_per_branch_eligible,presyn_dendrite_skeletal_length,presyn_dendrite_area,presyn_dendrite_mesh_volume,presyn_dendrite_n_branches,presyn_axon_skeletal_length,presyn_axon_area,presyn_axon_mesh_volume,presyn_axon_n_branches,presyn_basal_skeletal_length,presyn_basal_area,presyn_basal_mesh_volume,presyn_basal_n_branches,presyn_apical_skeletal_length,presyn_apical_area,presyn_apical_mesh_volume,presyn_apical_n_branches,presyn_apical_tuft_skeletal_length,presyn_apical_tuft_area,presyn_apical_tuft_mesh_volume,presyn_apical_tuft_n_branches,presyn_apical_shaft_skeletal_length,presyn_apical_shaft_area,presyn_apical_shaft_mesh_volume,presyn_apical_shaft_n_branches,presyn_oblique_skeletal_length,presyn_oblique_area,presyn_oblique_mesh_volume,presyn_oblique_n_branches,presyn_apical_total_skeletal_length,presyn_apical_total_area,presyn_apical_total_mesh_volume,presyn_apical_total_n_branches,presyn_n_syn_valid,presyn_n_syn_valid_pre,presyn_n_syn_valid_post,presyn_n_syn_error,presyn_n_syn_error_pre,presyn_n_syn_error_post,presyn_n_syn_presyns_on_dendrite,presyn_n_syn_mesh_errored,presyn_n_syn_distance_errored,presyn_n_syn_no_label,presyn_n_syn_head,presyn_n_syn_neck,presyn_n_syn_shaft,presyn_n_syn_no_head,presyn_n_syn_bouton,presyn_n_syn_non_bouton,presyn_n_syn_dendrite,presyn_n_syn_axon,presyn_n_syn_basal,presyn_n_syn_apical,presyn_n_syn_apical_tuft,presyn_n_syn_apical_shaft,presyn_n_syn_oblique,presyn_n_syn_soma,presyn_n_syn_apical_total,presyn_n_syn_dendrite_head_postsyn,presyn_n_syn_dendrite_neck_postsyn,presyn_n_syn_dendrite_shaft_postsyn,presyn_n_syn_dendrite_no_head_postsyn,presyn_n_syn_axon_bouton_presyn,presyn_n_syn_axon_bouton_postsyn,presyn_n_syn_axon_non_bouton_presyn,presyn_n_syn_axon_non_bouton_postsyn,presyn_n_syn_basal_head_postsyn,presyn_n_syn_basal_nec

In [83]:
import datajoint_utils as du

In [ ]:
du.mi

In [80]:
segment_ids = ["864691135700263803_0","864691135210057664_0"]
segment_ids = ["864691135334530025_1","864691136723462909_0"]
segment_ids = ["864691135269921701_0","864691135753726925_1"]

In [81]:
import proofread_verification as pv
pv.plot_proofread_neuron(segment_ids[0],plot_error_mesh=False)

Synapse Colors:
  head:red
  neck:aqua
  shaft:plum
  no_head:limegreen
  bouton:greenyellow
  non_bouton:aqua
  no_label:royalblue

Compartment Colors:
  valid_mesh:green
  axon:red
  dendrite:lightsteelblue


In [82]:
import microns_graph_utils as mgu

mgu.visualize_graph_connections_by_method(
G,
    segment_ids = segment_ids,
    #synapse_ids=syn_ids,
    method = "meshAfterParty",
    plot_synapse_skeletal_paths = True,
    verbose = False,
)

segment_ids_colors = ['blue', 'green']

Compartment Colors:
  valid_mesh:green


# Checking Soma Inhibition

In [42]:
import pandas_utils as pu
pu.set_pd_display_no_truncate_col()
edge_df_filt.query("(postsyn_compartment == 'soma_nan') and (presyn_cell_type == 'excitatory') " )[["presyn","postsyn","postsyn_cell_type"]]

,presyn,postsyn,postsyn_cell_type
5236,864691135974639471_0,864691135366598130_0,inhibitory
25818,864691135534710980_0,864691135307081798_0,inhibitory
25826,864691135534710980_0,864691135994543018_0,inhibitory
30919,864691136065684120_0,864691135474697792_0,excitatory
30920,864691136065684120_0,864691135474697792_0,excitatory
...,...,...,...
1903667,864691135474657088_0,864691135494029712_0,excitatory
1903668,864691135474657088_0,864691135494029712_0,excitatory
1903879,864691137019571822_0,864691136123743782_0,inhibitory
1903892,864691136617172187_2,864691135759723709_0,excitatory


In [65]:
segment_ids = ["864691137019571822_0","864691136123743782_0"] # looks good
segment_ids = ["864691136617172187_2","864691135759723709_0"] # looks good
segment_ids = ["864691136065684120_0","864691135474697792_0"] # maybe not
segment_ids = ["864691135474657088_0","864691135494029712_0"]


In [66]:
import proofread_verification as pv
pv.plot_proofread_neuron(segment_ids[0],plot_error_mesh=False)

Synapse Colors:
  head:red
  neck:aqua
  shaft:plum
  no_head:limegreen
  bouton:greenyellow
  non_bouton:aqua
  no_label:royalblue

Compartment Colors:
  valid_mesh:green
  apical_tuft:purple
  apical_shaft:aqua
  oblique:green
  apical:blue
  basal:yellow
  axon:red


In [67]:
import proofread_verification as pv
pv.plot_proofread_neuron(segment_ids[1],plot_error_mesh=False)

Synapse Colors:
  head:red
  neck:aqua
  shaft:plum
  no_head:limegreen
  bouton:greenyellow
  non_bouton:aqua
  no_label:royalblue

Compartment Colors:
  valid_mesh:green
  apical_tuft:purple
  apical_shaft:aqua
  oblique:green
  apical:blue
  basal:yellow
  axon:red


In [68]:
import microns_graph_utils as mgu

mgu.visualize_graph_connections_by_method(
G,
    segment_ids = segment_ids,
    #synapse_ids=syn_ids,
    method = "meshAfterParty",
    plot_synapse_skeletal_paths = True,
    verbose = False,
)

segment_ids_colors = ['blue', 'green']

Compartment Colors:
  valid_mesh:green


In [55]:
import microns_graph_utils as mgu

mgu.visualize_graph_connections_by_method(
G,
    segment_ids = segment_ids,
    #synapse_ids=syn_ids,
    method = "neuroglancer",
    plot_synapse_skeletal_paths = True,
    verbose = False,
)

segment_ids_colors = ['blue', 'green']


In [ ]:
"""
Conclusion: For most of the excitatory 
    --> soma synapses seems to maybe not right and result of error


"""